In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load data
us_df = pd.read_csv('../data/trajectories/steady_state/control_inputs_uniform.csv')
ys_df = pd.read_csv('../data/trajectories/steady_state/observations_steady_state_src_demo_17oct24.csv')
print(us_df.head())
print(ys_df.head())

In [ ]:
# We center the data, assuming first data point is the rest (vertical) position
# ys_df = ys_df - ys_df.iloc[0]
rest_positions = np.array([0.1005, -0.10698, 0.10445, -0.10302, -0.20407, 0.10933, 0.10581, -0.32308, 0.10566])
ys_df = ys_df - rest_positions

In [ ]:
def set_axes_equal(ax):
    """
    Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    """

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

In [ ]:
# Visualize data points
plt.close('all')

# Enable the ipympl backend for interactive plots
# %matplotlib widget

z_dataframes = [ys_df]
position_colors = ['r', 'b', 'g']

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for z_df in z_dataframes:
    x1, y1, z1 = z_df['x1'], z_df['y1'], z_df['z1']
    x2, y2, z2 = z_df['x2'], z_df['y2'], z_df['z2']
    x3, y3, z3 = z_df['x3'], z_df['y3'], z_df['z3']
    ax.scatter(x1, y1, z1, color=position_colors[0], label='Position 1', alpha=0.6)
    ax.scatter(x2, y2, z2, color=position_colors[1], label='Position 2', alpha=0.6)
    ax.scatter(x3, y3, z3, color=position_colors[2], label='Position 3', alpha=0.6)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.legend()
set_axes_equal(ax)

In [ ]:
# Convert to numpy
ys = ys_df.to_numpy()
n = ys.shape[0]
us = us_df.to_numpy()[:n, 1:]

# Split in train and test data
n_train = int(n * 0.85)
n_test = n - n_train

us_train = us[:n_train]
us_test = us[n_train:]

ys_train = ys[:n_train]
ys_test = ys[n_train:]

# Find mapping $G$ from $y$ to $u$

In [ ]:
# Simple least squares to find G in u = G*y
G = np.linalg.lstsq(ys_train, us_train, rcond=None)[0].T

# Save G
np.save('../data/models/ik/y2u_src_demo.npy', G)

In [ ]:
# Predict u_test using the learned G
us_pred = ys_test @ G.T

In [ ]:
# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(np.mean(np.square(us_test - us_pred)))
print(f"Root Mean Squared Error on the test data: {rmse}")

In [ ]:
# Mean absolute error
mae = np.mean(np.abs(us_test - us_pred))
print(f"Mean Absolute Error on the test data: {mae}")

In [ ]:
# For sense of scale, calculate the mean of |us|
mean_control_input = np.mean(np.abs(us))
print(f"Mean of the control inputs: {mean_control_input}")

# Find mapping $H$ from $u$ to $y$

In [ ]:
# Simple least squares to find H in u = G*y
H = np.linalg.lstsq(us_train, ys_train, rcond=None)[0].T

# Save H
np.save('../data/models/ik/u2y.npy', H)

In [ ]:
ys_pred = us_test @ H.T

In [ ]:
ys_pred.shape

In [ ]:
rmse = np.sqrt(np.mean(np.square(ys_test - ys_pred)))
print(f"Root Mean Squared Error on the test data: {rmse}")

In [ ]:
# Visualize data points
plt.close('all')

# Enable the ipympl backend for interactive plots
%matplotlib widget

# position_colors = ['r', 'b', 'g']

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in range(len(ys_pred)):
    # x1, y1, z1, x2, y2, z2, x3, y3, z3 = ys_test[i]
    # ax.scatter(x1, y1, z1, color=position_colors[0], label='Test', alpha=0.6)
    # x1, y1, z1, x2, y2, z2, x3, y3, z3 = ys_pred[i]
    # ax.scatter(x1, y1, z1, color=position_colors[1], label='Pred', alpha=0.6)
    x1, y1, z1, x2, y2, z2, x3, y3, z3 = ys_test[i]
    ax.scatter(x2, y2, z2, color=position_colors[0], label='Test', alpha=0.6)
    x1, y1, z1, x2, y2, z2, x3, y3, z3 = ys_pred[i]
    ax.scatter(x2, y2, z2, color=position_colors[1], label='Pred', alpha=0.6)


    # ax.scatter(x2, y2, z2, color=position_colors[1], label='Position 2', alpha=0.6)
    # ax.scatter(x3, y3, z3, color=position_colors[2], label='Position 3', alpha=0.6)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
# ax.legend()
set_axes_equal(ax)